In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.data import FragmentIonIntensityDataset

import os

2024-10-29 19:53:32.440871: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 19:53:32.487010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 19:53:32.487063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 19:53:32.488723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 19:53:32.497200: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 19:53:32.500521: I tensorflow/core/platform/cpu_feature_guard.cc:1


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [5]:
int_data = FragmentIonIntensityDataset.load_from_disk("/cmnfs/proj/prosit_astral/datasets/_dlomix_saved/full_dataset")

In [2]:
model_name = 'f_r_single_both_d5_static_0.0005_GTs'     # full
# model_name = 'b_r_single_both_d5_static_0.0005_TeG'   % balanced


model = keras.saving.load_model('/cmnfs/proj/prosit_astral/saved_models/final_models/%s.keras' % model_name,
                                custom_objects={'masked_spectral_distance': masked_spectral_distance, 'masked_pearson_correlation_distance': masked_pearson_correlation_distance}, 
                                compile=True, safe_mode=True)
model.summary()

Model: "transformer_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  6656      
                                                                 
 dense_1 (Dense)             multiple                  3328      
                                                                 
 dense_2 (Dense)             multiple                  131584    
                                                                 
 trans_block (TransBlock)    multiple                  524545    
                                                                 
 trans_block_1 (TransBlock)  multiple                  524545    
                                                                 
 trans_block_2 (TransBlock)  multiple                  524545    
                                                                 
 trans_block_3 (TransBlock)  multiple            

In [7]:
len(int_data.tensor_train_data)

12378

In [20]:
losses = []

eval_batches = [m for m in int_data.tensor_test_data.take(50)]

print("data loaded")

pred = []
true = []

for j in range(0, len(eval_batches)):

    predictions = model.predict(eval_batches[j][0], verbose=0)

    pred += np.split(predictions,        predictions.shape[0], axis=0) 
    true += np.split(eval_batches[j][1], predictions.shape[0], axis=0) 

    if j % 10 == 0:
        print(j)

pred = np.concatenate(pred, axis=0)
true = np.concatenate(true, axis=0)

print('Loss: ', tf.reduce_mean( masked_spectral_distance(true, pred) ))

losses.append(masked_spectral_distance(true, pred))

print(losses)

data loaded
0
10
20
30
40
Loss:  tf.Tensor(0.102522925, shape=(), dtype=float32)
[<tf.Tensor: shape=(51200,), dtype=float32, numpy=
array([0.03665632, 0.26280963, 0.05752968, ..., 0.08035967, 0.02115359,
       0.2068801 ], dtype=float32)>]


In [24]:
losses

# get median of tensor losses
np.median(np.array(losses), axis=0)

[<tf.Tensor: shape=(51200,), dtype=float32, numpy=
 array([0.03665632, 0.26280963, 0.05752968, ..., 0.08035967, 0.02115359,
        0.2068801 ], dtype=float32)>]

In [18]:
[i for i in int_data.tensor_test_data.take(1)][0][0]

{'modified_sequence': <tf.Tensor: shape=(1024, 30), dtype=int64, numpy=
 array([[ 5, 16, 14, ...,  0,  0,  0],
        [ 9, 20, 20, ...,  0,  0,  0],
        [16, 18,  4, ...,  0,  0,  0],
        ...,
        [10, 13, 10, ...,  0,  0,  0],
        [19,  1,  4, ...,  0,  0,  0],
        [ 4, 17, 13, ...,  0,  0,  0]])>,
 'charge_oh': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]], dtype=float32)>,
 'collision_energy': <tf.Tensor: shape=(1024,), dtype=float32, numpy=
 array([0.29031748, 0.33366817, 0.3347874 , ..., 0.31318372, 0.21907659,
        0.3157811 ], dtype=float32)>,
 'method_nr_oh': <tf.Tensor: shape=(1024, 2), dtype=float32, numpy=
 array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=f

In [23]:
# save tensor to disk as csv
np.savetxt('plots/%s_predictions.csv' % model_name, losses, delimiter='\n')